In [25]:
# 함수형 API를 사용하는 방법

In [26]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [27]:
# 1) 컬럼명 정의
cols = [
    "area", "perimeter", "compactness",
    "length", "width", "asymmetry", "groove_length",
    "target"
]

In [28]:
# 2) seeds_dataset 로드
df = pd.read_csv('./seeds_dataset.txt', sep=r"\s+", names=cols)

# 3) 특성(X)과 레이블(y) 분리
X = df.drop("target", axis=1).values      # shape (210, 7)
y = df["target"].values - 1               # 원래 1,2,3 → 0,1,2

# 4) 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [29]:
# 5) 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

In [30]:
# 6) 텐서 변환
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.int32)
X_test  = tf.convert_to_tensor(X_test,  dtype=tf.float32)
y_test  = tf.convert_to_tensor(y_test,  dtype=tf.int32)

In [31]:
# 7) Functional API 모델 정의
inputs     = Input(shape=(7,), name="input")                  # 7개 피처
x          = Dense(units=16, activation='relu', name="h1")(inputs)
x          = Dense(units=8,  activation='relu', name="h2")(x)
prediction = Dense(units=3,  activation='softmax', name="out")(x)

model = Model(inputs=inputs, outputs=prediction, name="seeds_func")

In [32]:
# 8) 컴파일
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [33]:
# 9) 학습
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=30,
    batch_size=16,
    verbose=1
)


Epoch 1/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - accuracy: 0.5638 - loss: 1.1275 - val_accuracy: 0.5882 - val_loss: 1.0868
Epoch 2/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.5678 - loss: 1.0818 - val_accuracy: 0.5882 - val_loss: 1.0137
Epoch 3/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.6601 - loss: 0.9446 - val_accuracy: 0.6471 - val_loss: 0.9478
Epoch 4/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6046 - loss: 0.9274 - val_accuracy: 0.6471 - val_loss: 0.8881
Epoch 5/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6316 - loss: 0.8850 - val_accuracy: 0.6471 - val_loss: 0.8343
Epoch 6/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6022 - loss: 0.8676 - val_accuracy: 0.7353 - val_loss: 0.7828
Epoch 7/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7871 - loss: 0.8099 - val_accuracy: 0.8235 - val_loss: 0.7365
Epoch 8/30
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.8602 - loss: 0.7816 - val_accuracy: 0.8824 - val_loss: 0.6960


In [34]:
# 10) 평가
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"테스트 정확도: {acc:.4f}")

테스트 정확도: 0.9048
